In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
data = ([1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21','2019-03-31', 30])

columns=['product_id', 'start_date', 'end_date', 'price']

prod_df = spark.createDataFrame(data = data, schema = columns)

data1 = ([1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30])

columns1 =['product_id', 'purchase_date', 'units']

unitssold_df = spark.createDataFrame(data = data1, schema = columns1)

In [0]:
prod_df.show()
unitssold_df.show()

+----------+----------+----------+-----+
|product_id|start_date|  end_date|price|
+----------+----------+----------+-----+
|         1|2019-02-17|2019-02-28|    5|
|         1|2019-03-01|2019-03-22|   20|
|         2|2019-02-01|2019-02-20|   15|
|         2|2019-02-21|2019-03-31|   30|
+----------+----------+----------+-----+

+----------+-------------+-----+
|product_id|purchase_date|units|
+----------+-------------+-----+
|         1|   2019-02-25|  100|
|         1|   2019-03-01|   15|
|         2|   2019-02-10|  200|
|         2|   2019-03-22|   30|
+----------+-------------+-----+



In [0]:
join_df = prod_df.join(unitssold_df,(prod_df.product_id == unitssold_df.product_id) & (unitssold_df.purchase_date >= prod_df.start_date) & (unitssold_df.purchase_date <= prod_df.end_date),"inner").select("*",((col("price"))*(col("units"))).alias("total_price"))

In [0]:
join_df.show()

+----------+----------+----------+-----+----------+-------------+-----+-----------+
|product_id|start_date|  end_date|price|product_id|purchase_date|units|total_price|
+----------+----------+----------+-----+----------+-------------+-----+-----------+
|         1|2019-02-17|2019-02-28|    5|         1|   2019-02-25|  100|        500|
|         1|2019-03-01|2019-03-22|   20|         1|   2019-03-01|   15|        300|
|         2|2019-02-01|2019-02-20|   15|         2|   2019-02-10|  200|       3000|
|         2|2019-02-21|2019-03-31|   30|         2|   2019-03-22|   30|        900|
+----------+----------+----------+-----+----------+-------------+-----+-----------+



In [0]:
join_df.groupBy(prod_df['product_id']).agg(round(((sum("total_price"))/(sum("units"))),2).alias("average_price")).show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+

